In [2]:
# Folowing are packasges need to install to run this program.
#pip install SpeechRecognition
#pip install pyttsx3
#pip install pywhatkit
#pip install wikipedia
#pip install pyjokes
#pip install jinja2
#pip install PyAudio
#pip install wolframalpha

In [3]:
#from markupsafe import escape
#pip install wolframalpha -- from anaconda propt

In [4]:
#pip install PyAudio-0.2.11-cp39-cp39-win_amd64.whl
# from https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyaudio
#pip install sentence_transformers

In [5]:
# importing libraries 
import pandas as pd
import pyttsx3
import datetime
import wikipedia
import requests

In [6]:
import speech_recognition as sr
import pyaudio
import pyjokes
import pywhatkit
from bs4 import BeautifulSoup

In [7]:
import webbrowser
import os
import random
import sys
import shutil

In [8]:
#pip install ecapture - from promt
#pip install twilio

In [11]:
import cv2
import wolframalpha
from twilio.rest import Client
#from clint.textui import progress
#from ecapture import ecapture as ec

In [12]:
import googletrans
from googletrans import Translator

In [13]:
def speak(audio):
    engine.say(audio)
    engine.runAndWait()
    

In [14]:
engine=pyttsx3.init('sapi5')
voices=engine.getProperty('voices')
print(voices[2].id)


HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0


In [15]:
engine.setProperty('voice',voices[2].id)

In [16]:
speak('how are you')

In [17]:
# this function will greet
def wishme():
    hour=int(datetime.datetime.now().hour)
    if hour>=0 and hour <12:
        speak('good morning')
    elif hour>=12 and hour<16:
        speak('good afternoon')
    elif hour >=16 and hour<19:
        speak('good evening')
    else:
        speak('good night')
    speak('I am VAI, your virtual artificial intelegence, how may i help you?')

In [18]:
wishme()

In [19]:
# It will take command from user
def takecommand():
    r=sr.Recognizer()

    with sr.Microphone() as source:
        print('listening..')
        r.pause_threshold=1
        audio=r.listen(source) 
    try:
        print('recognizing')
        query=r.recognize_google(audio,language='en-in')
        print(f'user said:{query}\n')
    except Exception as e:
        print(e)
        print('say that again please')
        return 'None'
    return query


In [20]:
"""Go to newsapi.org
Create account where you will get api key
paste in url your api key
"""

def news():
    main_url='https://newsapi.org/v2/top-headlines?sources=techcrunch&apikey=c03e1581b08d4921b5d082aef0a11b80'
    main_page=requests.get(main_url).json()
    #print(main_page)
    articles=main_page['articles']
    #print(articles)
    head=[]
    day=['first','second','third','fourth','fifth']
    for ar in articles:
        head.append(ar['title'])
    for i in range(len(day)):
        print(f'todays {day[i]} news is :{head[i]}')
        speak(f'todays {day[i]} news is :{head[i]}')

In [21]:
#news()

In [22]:
%%time
from sentence_transformers import SentenceTransformer,util

KeyboardInterrupt: 

In [23]:
%%time
# load transformer models
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
df=pd.read_csv('question_embedding.csv')

NameError: name 'SentenceTransformer' is not defined

In [ ]:
def question_search(ques, emmbding_df, conf):
    """
    Obj: This function is used to check the similar question from the databases
    useing embeddings.
    :param ques: input question
    :param emmbding_df: data frame which contains embeddings for all the question
    :param conf: confidence
    :return: list of similar questions
    """
    ques = [ques]
    # extract embeddings for  all question
    sentence_embd = emmbding_df["embd"].to_list()
    # create embeddings for questions
    question_embd = model.encode(ques, convert_to_tensor=True)

    # Compute cosine-similarities for each sentence with each other sentence
    cosine_scores = util.cos_sim(sentence_embd, question_embd)

    emmbding_df["score"] = [round(float(score), 2) for score in cosine_scores]  # remove
    result = emmbding_df[emmbding_df["score"] >= conf].reset_index(drop=True)
    # convert all question into list
    sq = result["Question"].to_list()
    return sq
df["embd"] = df["embd"].apply(lambda x: eval(x))

In [ ]:
# df["embd"] = df["embd"].apply(lambda x: eval(x))

In [24]:
""" We can give command like search in wikipedia, open youtube and play music, play song in youtube.
we can ask current time,jokes and translate in any language, I have selected Hindi language.
we can extract similarity snetences from embeded dataframe using cosine similarity.
"""
while True:
    query=takecommand().lower()

    if  'who is' in query:
        speak('searching wikipedia...')
        query=query.replace('wikipedia','')
        results=wikipedia.summary(query,sentences=2)
        speak('according to wikipedia')
        print(results)
        speak(results)
    elif 'open youtube' in query:
        webbrowser.open('youtube.com')
    elif 'open google' in query:
        webbrowser.open('google.com')
    elif 'play music' in query:
        music_dir='D:\\songs'
        songs=os.listdir(music_dir)
        rd=random.choice(songs)
        os.startfile(os.path.join(music_dir,rd))
        print(songs)
        os.startfile(os.path.join(music_dir,songs[0]))
    elif 'time' in query:
        strTime=datetime.datetime.now().strftime('%H:%M:%S')
        speak(f'time is {strTime}')
    elif 'play' in query:
        song = query.replace('play', '')
        speak (song)
        pywhatkit.playonyt(song)
    elif 'joke' in query:
        jokes=(pyjokes.get_joke())
        print(jokes)
        speak(jokes) 
    elif 'similarity' in query:
        que=query.replace('similarity','')
        print(que)
        res = question_search(que, df, conf=0.40)
        #sq = res["Question"].to_list()
        print("Similar questions found...")
        for i in res:
            print(i)
            speak(i)
        
    elif 'temperature' in query:
        speak('searching temperature')
        temp='temperature in mumbai'
        url=f'https://google.com/search?q={temp}'
        rs=requests.get(url)
        data=BeautifulSoup(rs.text,'html.parser' )
        temps=data.find('div', class_='BNeawe').text
        print(temps)
        speak(f'currenr {temp} is {temps}')
    elif 'translate' in query:
        translator=Translator()
        sen=query.replace('translate','')
        sen=translator.translate(sen,src='en',dest='hi')
        print(sen)
        speak(sen.pronunciation)
    elif 'send message' in query:
        pywhatkit.sendwhatmsg_instantly('+919820676422','this is test message')
    elif 'exit' in query:
        break
    elif 'open camera' in query:
        cap=cv2.VideoCapture(0)
        while True:
            ret,img=cap.read()
            cv2.imshow('webcam',img)
            if cv2.waitKey(1)& 0xFF==ord('q'):
           # k=cv2.waitKey(50)
            #if k==27:
            # press q foe quit
                break;
        cap.release()
        cv2.destroyAllWindows()
    elif 'news' in query:
        news()
    elif "don't listen" in query or "stop listening" in query:
            speak("for how much time you want to stop VAI from listening commands")
            a = int(takeCommand())
            time.sleep(a)
            print(a)
    elif "where is" in query:
            query = query.replace("where is", "")
            location = query
            speak("User asked to Locate")
            speak(location)
            webbrowser.open("https://www.google.com/maps/place/'" + location + "")
            #('https://www.google.com/maps/place/' + map_string)
    elif "calculate" in query:
             
            app_id = "5QVV3E-GYRWXY8UV5"
            client = wolframalpha.Client(app_id)
            indx = query.lower().split().index('calculate')
            query = query.split()[indx + 1:]
            res = client.query(' '.join(query))
            answer = next(res.results).text
            print("The answer is " + answer)
            speak("The answer is " + answer)
"""wolframalpha- we can calculate, find GDP ,weather,conversion almost anything we can ask from it.
"""

#APPID: 5QVV3E-GYRWXY8UV5
     
        

listening..
recognizing
user said:open camera

listening..
recognizing
user said:make sentence using automatic

listening..
recognizing

say that again please
listening..
recognizing
user said:I like how did you come on this

listening..
recognizing

say that again please
listening..
recognizing

say that again please
listening..
recognizing
user said:don't listen



Exception ignored in: <function BSTR.__del__ at 0x000001AA9CB190D0>
Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\lib\site-packages\comtypes\__init__.py", line 999, in __del__
    def __del__(self, _free=windll.oleaut32.SysFreeString):
KeyboardInterrupt: 


NameError: name 'takeCommand' is not defined

In [ ]:
#<div jscontroller="e0Sh5" class="Ab33Nc" aria-level="3" role="heading"><div><div class="vk_bk TylWce SGNhVe"><span class="wob_t q8U8x" id="wob_tm" sty
# <div class="vk_bk TylWce SGNhVe"><span class="wob_t q8U8x" id="wob_tm" style="display:inline">27</span><span class="wob_t" id="wob_ttm" style="display:none">81</span></div>

In [ ]:
df=pd.read_csv('question_embedding.csv')

In [ ]:
%%time
from sentence_transformers import SentenceTransformer

In [6]:
def speak(audio):
	engine.say(audio)
	engine.runAndWait()

def wishMe():
	hour = int(datetime.datetime.now().hour)
	if hour>= 0 and hour<12:
		speak("Good Morning Sir !")

	elif hour>= 12 and hour<18:
		speak("Good Afternoon Sir !")

	else:
		speak("Good Evening Sir !")

	assname =("Jarvis 1 point o")
	speak("I am your Assistant")
	speak(assname)
	

def username():
	speak("What should i call you sir")
	uname = takeCommand()
	speak("Welcome Mister")
	speak(uname)
	columns = shutil.get_terminal_size().columns
	
	print("#####################".center(columns))
	print("Welcome Mr.", uname.center(columns))
	print("#####################".center(columns))
	
	speak("How can i Help you, Sir")

def takeCommand():
	
	r = sr.Recognizer()
	
	with sr.Microphone() as source:
		
		print("Listening...")
		r.pause_threshold = 1
		audio = r.listen(source)

	try:
		print("Recognizing...")
		query = r.recognize_google(audio, language ='en-in')
		print(f"User said: {query}\n")

	except Exception as e:
		print(e)
		print("Unable to Recognize your voice.")
		return "None"
	
	return query

def sendEmail(to, content):
	server = smtplib.SMTP('smtp.gmail.com', 587)
	server.ehlo()
	server.starttls()
	
	# Enable low security in gmail
	server.login('your email id', 'your email password')
	server.sendmail('your email id', to, content)
	server.close()


In [13]:
username()

Listening...
Recognizing...
User said: Alpha Alpha

                                                 #####################                                                  
Welcome Mr.                                                       Alpha Alpha                                                       
                                                 #####################                                                  


In [ ]:
%%time
# load transformer models
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
def question_search(ques, emmbding_df, conf):
    """
    Obj: This function is used to check the similar question from the databases
    useing embeddings.
    :param ques: input question
    :param emmbding_df: data frame which contains embeddings for all the question
    :param conf: confidence
    :return: list of similar questions
    """
    ques = [ques]
    # extract embeddings for  all question
    sentence_embd = emmbding_df["embd"].to_list()
    # create embeddings for questions
    question_embd = model.encode(ques, convert_to_tensor=True)

    # Compute cosine-similarities for each sentence with each other sentence
    cosine_scores = util.cos_sim(sentence_embd, question_embd)

    emmbding_df["score"] = [round(float(score), 2) for score in cosine_scores]  # remove
    result = emmbding_df[emmbding_df["score"] >= conf].reset_index(drop=True)
    # convert all question into list
    sq = result["Question"].to_list()
    return sq

In [ ]:
#df["embd"] = df["embd"].apply(lambda x: eval(x))

In [ ]:
res = question_search(que, df, conf=0.40)
#sq = res["Question"].to_list()
print("Similar questions found...")
for i in res:
    print(i)
    speak(i)

In [ ]:
def translate(text):
    translator=Translator()
    sen=translator.translate(text,src='en',dest='hi')
    print(sen.text)
    return sen

In [ ]:
a=translator.translate('आप कैसे हैं')
print(a.text)


In [ ]:
# from google_trans_new import google_translator  
  
# translator = google_translator()  
# translate_text = translator.translate('Hola mundo!', lang_src='es', lang_tgt='en')  
# print(translate_text)

In [ ]:
# temp='temperture in mubai'
# url=f'https://google.com/search?q={temp}'
# rs=requests.get(url)
# data=BeautifulSoup(rs.text,'html.parser' )
# temps=data.find('div', class_='BNeawe').text
# print(temps)
# speak(f'current {temp} is {temps}')